In [1]:
# Template from Ruoyi in the email
# Friday, July 31, 2020 at 3:26 PM

import os
import datetime
import pandas as pd
import numpy as np
import glob

print(datetime.datetime.now())

2020-08-03 10:09:46.399565


In [2]:
date_quarter_start=datetime.date(2020,8,2)
date_quarter_end=date_quarter_start+datetime.timedelta(days=13*7-1)
days_open_in_x=21

folder_store_list_Dom="/home/jian/BigLots/static_files/store_list_from_Dom/"
folder_store_list_IT="/home/jian/BigLots/static_files/Store_list/"
folder_store_quadrant_mapping="/home/jian/Projects/Big_Lots/New_TA/package_to_run_TA_every_quarter/quadrants_mapping/"
folder_store_competitor_mapping="/home/jian/Projects/Big_Lots/New_TA/package_to_run_TA_every_quarter/competitor_mapping/"

def get_latest_file_in_folder(folder):
    list_all_files=glob.glob(folder+"*")
    list_all_files=sorted(list_all_files,key=lambda x: os.stat(x).st_mtime)
    return list_all_files[-1]

In [3]:
excel_store_list_Dom=get_latest_file_in_folder(folder_store_list_Dom)
print("excel_store_list_Dom: \n",excel_store_list_Dom,"\n")

txt_store_list_IT=get_latest_file_in_folder(folder_store_list_IT)
print("txt_store_list_IT: \n",txt_store_list_IT,"\n")

excel_store_quadrant_mapping=get_latest_file_in_folder(folder_store_quadrant_mapping)
print("excel_store_quadrant_mapping: \n",excel_store_quadrant_mapping,"\n")

excel_store_competitor_mapping=get_latest_file_in_folder(folder_store_competitor_mapping)
print("excel_store_competitor_mapping: \n",excel_store_competitor_mapping,"\n")

excel_store_list_Dom: 
 /home/jian/BigLots/static_files/store_list_from_Dom/Store List Report 07.10.20 425PM.xlsx 

txt_store_list_IT: 
 /home/jian/BigLots/static_files/Store_list/MediaStormStores20200801-134534-635.txt 

excel_store_quadrant_mapping: 
 /home/jian/Projects/Big_Lots/New_TA/package_to_run_TA_every_quarter/quadrants_mapping/location_id_20q1_quadrant.xlsx 

excel_store_competitor_mapping: 
 /home/jian/Projects/Big_Lots/New_TA/package_to_run_TA_every_quarter/competitor_mapping/Biglots_Competitors_PS_Count by store_20200731 CC.xlsx 



In [4]:
df_store_list_dom=pd.ExcelFile(excel_store_list_Dom)
df_store_list_dom=df_store_list_dom.parse("Store List",dtype=str,
                                         usecols=['Store','Address 1','City','ST','Zip','SOTF','Furniture Type','Open Date'])
list_SOTF=df_store_list_dom[df_store_list_dom['SOTF'].apply(lambda x: str(x).lower()=="yes")]['Store'].tolist()
df_store_list_dom['Open Date']=df_store_list_dom['Open Date'].apply(lambda x: str(x).split(" ")[0])

In [5]:
# 
ind_checkpoint_1=df_store_list_dom[df_store_list_dom['Store']=="New or Relo Locations"].index[0]
df_1_currently_open=df_store_list_dom.iloc[:ind_checkpoint_1,]
df_2_to_open=df_store_list_dom.iloc[ind_checkpoint_1:,].reset_index()
del df_2_to_open['index']
# 
ind_checkpoint_2=df_2_to_open[pd.isnull(df_2_to_open['Store'])].index[0]
df_2_to_open=df_2_to_open.iloc[:ind_checkpoint_2,]
df_2_to_open=df_2_to_open[pd.notnull(df_2_to_open['Address 1'])]


In [6]:
len(list_SOTF)

491

In [7]:
df_1_currently_open=df_1_currently_open[pd.notnull(df_1_currently_open['Store'])]
df_2_to_open=df_2_to_open[pd.notnull(df_2_to_open['Store'])]

df_2_to_open['Open Date']=pd.to_datetime(df_2_to_open['Open Date']).dt.date

In [8]:
df_2_to_open=df_2_to_open[df_2_to_open['Open Date']<=date_quarter_end]
df_2_to_open=df_2_to_open[df_2_to_open['Open Date']<=datetime.datetime.now().date()+datetime.timedelta(days=days_open_in_x)]
df_output=df_1_currently_open.append(df_2_to_open)
df_output['SOTF']=np.where(df_output['Store'].isin(list_SOTF),"Yes","Legacy")
df_output['Furniture Type']=df_output['Furniture Type'].fillna("FULLFURN")
df_output['Furniture Type']=df_output['Furniture Type'].replace("RTA0700","FULLFURN").replace("RTA1400","FULLFURN")

In [9]:
df_store_list_IT=pd.read_table(txt_store_list_IT,dtype=str,sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
df_store_list_IT=df_store_list_IT[['location_id','latitude_meas','longitude_meas']]
df_store_list_IT=df_store_list_IT.rename(columns={'location_id':"Store","latitude_meas":"lat","longitude_meas":"lng"})
df_store_list_IT['lat']=df_store_list_IT['lat'].astype(float)
df_store_list_IT['lng']=df_store_list_IT['lng'].astype(float)
df_store_list_IT.head(2)

,Store,lat,lng
0,3,33.421570,-82.018921
1,30,41.363132,-83.650909


In [10]:
df_quadrant=pd.read_excel(excel_store_quadrant_mapping,dtype=str).rename(columns={"location_id":"Store"})
df_competitor=pd.ExcelFile(excel_store_competitor_mapping)
df_competitor=df_competitor.parse("Competitor Rank",dtype=str)
df_competitor=df_competitor[['location_id','Competitor Rank']].rename(columns={"location_id":"Store","Competitor Rank":"Competitor_Rank"})

df_output=pd.merge(df_output,df_quadrant,on="Store",how="left")
df_output=pd.merge(df_output,df_competitor,on="Store",how="left")
df_output=pd.merge(df_output,df_store_list_IT,on="Store",how="left")

In [11]:
df_output['Furniture Type']=df_output['Furniture Type'].fillna("nan")
df_output['Quadrant']=df_output['Quadrant'].fillna("nan")
df_output['Competitor_Rank']=df_output['Competitor_Rank'].fillna("nan")
df_output['Zip']=df_output['Zip'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5)[:5] if type(x)==str else x)

In [12]:
df_output=df_output.reset_index()
del df_output['index']
for ind,row in df_output.iterrows():
    if pd.isnull(row['lat']) or row['lat']==0:
        print("store lat/lng' missing: %s"%row['Store'])
        search_key=" ".join([row['Address 1'],row['City'],row['ST'],row['Zip']])
        print(search_key)
        google_loc_pair=str(input()).replace(" ","")
        lat=float(google_loc_pair.split(",")[0])
        lng=float(google_loc_pair.split(",")[1])
        df_output.loc[ind,'lat']=lat
        df_output.loc[ind,'lng']=lng
        

store lat/lng' missing: 4516
755 W CRAIG RD NORTH LAS VEGAS NV 89031
36.2379132,-115.1539617
store lat/lng' missing: 4715
4525 N ORACLE ROAD TUSCON AZ 85705
32.2886211,-110.9820104
store lat/lng' missing: 4718
1631 WEST CRAIG ROAD NORTH LAS VEGAS NV 89032
36.2376368,-115.1683213
store lat/lng' missing: 4721
400 MAIN ST DENNIS PORT MA 02639
41.6668977,-70.1350297
store lat/lng' missing: 5416
260 REMOUNT ROAD FRONT ROYAL VA 22630
38.9090455,-78.1851776
store lat/lng' missing: 5422
433 COLUMBIA AVE LEXINGTON SC 29072
33.9881797,-81.2428385
store lat/lng' missing: 4716
4208 E BLUE GRASS RD MOUNT PLEASANT MI 48858
43.5741276,-84.7663118
store lat/lng' missing: 4717
360 EAST 10TH ST GILROY EPA ID, BAG BAN CA 95020
37.0001319,-121.5630096
store lat/lng' missing: 5418
1550 COSHOCTON AVE MOUNT VERNON OH 43050
40.4000587,-82.4448439
store lat/lng' missing: 5427
3010 NAVARRE AVE OREGON OH 43616
41.6350639,-83.4781977
store lat/lng' missing: 5428
12295 CAPITAL BLVD WAKE FOREST NC 27587
35.981846,-

In [13]:
'''
store lat/lng' missing: 4516
755 W CRAIG RD NORTH LAS VEGAS NV 89031
36.2379132,-115.1539617
store lat/lng' missing: 4715
4525 N ORACLE ROAD TUSCON AZ 85705
32.2886211,-110.9820104
store lat/lng' missing: 4718
1631 WEST CRAIG ROAD NORTH LAS VEGAS NV 89032
36.2376368,-115.1683213
store lat/lng' missing: 4721
400 MAIN ST DENNIS PORT MA 02639
41.6668977,-70.1350297
store lat/lng' missing: 5416
260 REMOUNT ROAD FRONT ROYAL VA 22630
38.9090455,-78.1851776
store lat/lng' missing: 5422
433 COLUMBIA AVE LEXINGTON SC 29072
33.9881797,-81.2428385
store lat/lng' missing: 4716
4208 E BLUE GRASS RD MOUNT PLEASANT MI 48858
43.5741276,-84.7663118
store lat/lng' missing: 4717
360 EAST 10TH ST GILROY EPA ID, BAG BAN CA 95020
37.0001319,-121.5630096
store lat/lng' missing: 5418
1550 COSHOCTON AVE MOUNT VERNON OH 43050
40.4000587,-82.4448439
store lat/lng' missing: 5427
3010 NAVARRE AVE OREGON OH 43616
41.6350639,-83.4781977
store lat/lng' missing: 5428
12295 CAPITAL BLVD WAKE FOREST NC 27587
35.981846,-78.5429906
store lat/lng' missing: 5430
3075 E 25TH ST COLUMBUS IN 47203
39.221862,-85.8887922
'''

"\nstore lat/lng' missing: 4516\n755 W CRAIG RD NORTH LAS VEGAS NV 89031\n36.2379132,-115.1539617\nstore lat/lng' missing: 4715\n4525 N ORACLE ROAD TUSCON AZ 85705\n32.2886211,-110.9820104\nstore lat/lng' missing: 4718\n1631 WEST CRAIG ROAD NORTH LAS VEGAS NV 89032\n36.2376368,-115.1683213\nstore lat/lng' missing: 4721\n400 MAIN ST DENNIS PORT MA 02639\n41.6668977,-70.1350297\nstore lat/lng' missing: 5416\n260 REMOUNT ROAD FRONT ROYAL VA 22630\n38.9090455,-78.1851776\nstore lat/lng' missing: 5422\n433 COLUMBIA AVE LEXINGTON SC 29072\n33.9881797,-81.2428385\nstore lat/lng' missing: 4716\n4208 E BLUE GRASS RD MOUNT PLEASANT MI 48858\n43.5741276,-84.7663118\nstore lat/lng' missing: 4717\n360 EAST 10TH ST GILROY EPA ID, BAG BAN CA 95020\n37.0001319,-121.5630096\nstore lat/lng' missing: 5418\n1550 COSHOCTON AVE MOUNT VERNON OH 43050\n40.4000587,-82.4448439\nstore lat/lng' missing: 5427\n3010 NAVARRE AVE OREGON OH 43616\n41.6350639,-83.4781977\nstore lat/lng' missing: 5428\n12295 CAPITAL BLV

In [14]:
output_folder="./store_mapping_output/"
try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)

In [15]:
writer=pd.ExcelWriter(output_folder+"bl_store_forAPP_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output.to_excel(writer,"store_mapping",index=False)
writer.save()

In [16]:
writer=pd.ExcelWriter(output_folder+"bl_store_forAPP_latest.xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"store_mapping",index=False)
writer.save()